## Week 1 
本周任務：
* 安裝 Anaconda 環境
* 註冊 github 帳號
* 初步探索 91APP 資料集

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
%matplotlib inline

In [3]:
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/91APPdataset/' 
data = pd.read_csv(path+'Orders.csv')

/home/kevin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 資料概覽

##### 取前三筆資料檢視其內容，確認此資料集是以商品為單位蒐集紀錄。

In [4]:
data.head(3).T

,0,1,2
DateId,20150703,20150703,20150703
MemberId,1388133,1388133,1388133
OrderGroupCode,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...
TrackSourceTypeDef,Web,Web,Web
TrackDeviceTypeDef,Mobile,Mobile,Mobile
PayProfileTypeDef,SevenEleven,SevenEleven,SevenEleven
SalesOrderSlaveId,4457777,4457778,4457779
SalePageId,1242871,1242880,1242880
IsMajor,True,True,True
IsGift,False,False,False


##### 羅列各變數的資料類型與數目
* 數值變數有16個，非數值變數有12個
* 自City變數以下的欄目存在缺失值

In [5]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5495276 entries, 0 to 5495275
Data columns (total 28 columns):
DateId                         5495276 non-null int64
MemberId                       5495276 non-null int64
OrderGroupCode                 5495276 non-null object
TrackSourceTypeDef             5495276 non-null object
TrackDeviceTypeDef             5495276 non-null object
PayProfileTypeDef              5495276 non-null object
SalesOrderSlaveId              5495276 non-null int64
SalePageId                     5495276 non-null int64
IsMajor                        5495276 non-null bool
IsGift                         5495276 non-null bool
IsSalePageGift                 5495276 non-null bool
Quantity                       5495276 non-null int64
UnitPrice                      5495276 non-null float64
PromotionDiscount              5495276 non-null float64
ECouponId                      5495276 non-null int64
ECouponDiscount                5495276 non-null float64
SalesOrderSlaveT

##### 移除編號變數後，檢視數值變數的統計值
* 主件佔全部商品紀錄的 97.6%, 贈品僅佔 2.4%
* 滿額贈品從未送出過
* 購買商品大多數為單件
* 單品平均與訂單明細金額平均差不多
* 折扣券折抵的金額較一般折抵來的少

In [6]:
numerics = ['bool', 'int64', 'float64']
numeric_columns = list(data.select_dtypes(include=numerics))
numeric_columns = [i for i in numeric_columns if not 'Id' in i and not 'Code' in i]
pd.set_option('display.float_format', lambda x: "%.3f" % x)
pd.DataFrame(data[numeric_columns], dtype='float64').describe()

,IsMajor,IsGift,IsSalePageGift,Quantity,UnitPrice,PromotionDiscount,ECouponDiscount,SalesOrderSlaveTotalPayment
count,5495276.000,5495276.000,5495276.000,5495276.000,5495276.000,5495276.000,5495276.000,5495276.000
mean,0.976,0.024,0.000,1.021,263.041,-25.132,-7.048,235.516
std,0.154,0.154,0.000,0.185,124.739,44.495,14.828,125.334
min,0.000,0.000,0.000,1.000,0.000,-3005.000,-207.000,0.000
25%,1.000,0.000,0.000,1.000,199.000,-31.000,-8.000,167.000
50%,1.000,0.000,0.000,1.000,237.000,0.000,0.000,199.000
75%,1.000,0.000,0.000,1.000,320.000,0.000,0.000,298.000
max,1.000,1.000,0.000,10.000,3500.000,0.000,0.000,14321.000


## 處理非數值資料
* 透過 Hot Key Encoding 轉換資料，加入原資料集
* 敘述統計

### TrackSourceTypeDef 訂單來源
* 從瀏覽器下單的客戶最多，達 57.7%
* 使用 Android 系統和 iOS 的用戶比例差不多，各約 21%

In [14]:
source = pd.get_dummies(data['TrackSourceTypeDef'], prefix="Source")
source.describe()

,Source_AndroidApp,Source_Web,Source_iOSApp
count,5495276.000,5495276.000,5495276.000
mean,0.212,0.577,0.211
std,0.409,0.494,0.408
min,0.000,0.000,0.000
25%,0.000,0.000,0.000
50%,0.000,1.000,0.000
75%,0.000,1.000,0.000
max,1.000,1.000,1.000


### TrackDeviceTypeDef 訂單裝置來源
* 透過智慧型手機購物的用戶居多，有 84.2%
* 使用電腦購物的用戶有 15.7%
* 幾乎沒有人用平板購物

In [15]:
device = pd.get_dummies(data["TrackDeviceTypeDef"], prefix="Device")
device.describe()

,Device_Mobile,Device_PC,Device_Pad
count,5495276.000,5495276.000,5495276.000
mean,0.842,0.157,0.000
std,0.364,0.364,0.007
min,0.000,0.000,0.000
25%,1.000,0.000,0.000
50%,1.000,0.000,0.000
75%,1.000,0.000,0.000
max,1.000,1.000,1.000


## PayProfileTypeDef 付款方式
* 90% 用戶習慣使用超商付款(7-11: 53.8% > 全家: 37.1%)
* 8.8% 的用戶以信用卡付款
* 用 ATM 和 LinePay 的客戶非常少

In [16]:
pay = pd.get_dummies(data["PayProfileTypeDef"], prefix="Pay")
pay.describe()

,Pay_ATM,Pay_CreditCardOnce,Pay_Family,Pay_LinePay,Pay_SevenEleven
count,5495276.000,5495276.000,5495276.000,5495276.000,5495276.000
mean,0.002,0.088,0.371,0.001,0.538
std,0.041,0.284,0.483,0.028,0.499
min,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.000,0.000,0.000,0.000
50%,0.000,0.000,0.000,0.000,1.000
75%,0.000,0.000,1.000,0.000,1.000
max,1.000,1.000,1.000,1.000,1.000


## City 訂單配送城市

In [18]:
#city = pd.get_dummies(data["City"], prefix="City")
city.describe().T

,count,mean,std,min,25%,50%,75%,max
City_1st,5495276.000,0.000,0.003,0.000,0.000,0.000,0.000,1.000
City_25,5495276.000,0.000,0.001,0.000,0.000,0.000,0.000,1.000
City_<fo,5495276.000,0.000,0.003,0.000,0.000,0.000,0.000,1.000
City_Aya,5495276.000,0.000,0.001,0.000,0.000,0.000,0.000,1.000
City_Cha,5495276.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000
City_Daf,5495276.000,0.000,0.001,0.000,0.000,0.000,0.000,1.000
City_Hsi,5495276.000,0.000,0.001,0.000,0.000,0.000,0.000,1.000
City_Hy,5495276.000,0.000,0.001,0.000,0.000,0.000,0.000,1.000
City_Ins,5495276.000,0.000,0.001,0.000,0.000,0.000,0.000,1.000
City_Jia,5495276.000,0.000,0.001,0.000,0.000,0.000,0.000,1.000


In [23]:
data.City.unique()

array(['嘉義縣', '桃園市', '台南市', '新北市', '新竹市', '台北市', '苗栗縣', '台中市', '高雄市',
       '彰化縣', '屏東縣', '新竹縣', '宜蘭縣', '花蓮縣', '雲林縣', '台東縣', '基隆市', '南投縣',
       '嘉義市', '台東市', '澎湖縣', '屏東市', 'New', '<fo', 'Kao', 'Kot', 'Tao',
       'Hy ', nan, '25 ', 'Keb', '1st', 'Tai', 'Ins', '桃園縣', 'Đại', 'Cha',
       'Xin', 'Quậ', 'Nan', 'No.', 'Hsi', 'Daf', 'Aya', 'Jia'], dtype=object)

,DateId,MemberId,OrderGroupCode,TrackSourceTypeDef,TrackDeviceTypeDef,PayProfileTypeDef,SalesOrderSlaveId,SalePageId,IsMajor,IsGift,...,SalesOrderReceiverId,City,District,ZipCode,StoreName,StoreId,DeliverTypeDef,StatusDef,ReturnGoodsDateId,CauseDef
470561,20170423,2867297,b'\xe1\xa4Z^\xc6\xebH\xb2\xc8\x91f\x8e]\\\xf6\...,Web,PC,SevenEleven,33711224,3138192,False,True,...,11577538,彰化縣,員林市,510.000,品冠門市,137878.000,SevenEleven,Cancel,nan,NaN
4924457,20161207,2278336,b'\x9b\x07\xc8\xad\xec\xa5\xd7\xbf\xd1\xe6\x97...,Web,Mobile,Family,25197609,2578672,True,False,...,8517239,彰化縣,員林市,510.000,全家員林車頭店,9336.000,Family,Finish,nan,NaN
1861505,20160405,3068259,b'\x11\x87\xc8y@\x1cV\x11\x89A4iu\xdcJ\xbc\x92...,AndroidApp,Mobile,Family,12308959,1922964,True,False,...,4107437,彰化縣,員林鎮,510.000,全家員林黎明店,13537.000,Family,Finish,nan,NaN
4648113,20161107,2229813,b'X\x04\x1d\x15\xb9\xe9I\x87\x83\xe57Y\x91\xcc...,iOSApp,Mobile,SevenEleven,22954490,2514489,True,False,...,7733321,彰化縣,員林市,510.000,員昌門市,110459.000,SevenEleven,Finish,nan,NaN
1578338,20160317,1095777,b'\xb8\xc0V\xb9\xe6\xadZ\xe6&\xc3\xc2\x06\x98\...,AndroidApp,Mobile,SevenEleven,11539815,1119387,True,False,...,3850109,彰化縣,員林鎮,510.000,員家門市,909552.000,SevenEleven,Finish,nan,NaN
2324703,20160524,319792,b'\xc3k\xea\xc0W\xc2)\xbb\x10\xadu\xb8\xd5\x8d...,Web,Mobile,SevenEleven,14513455,2146851,True,False,...,4840447,彰化縣,員林市,510.000,新百川門市,954459.000,SevenEleven,Finish,nan,NaN
1070450,20170604,2920942,b'\xfdG\xfa|\xb1YP/\xc1=\xfaI\xcc7\xa8\xda\xa8...,Web,PC,SevenEleven,36685837,3252103,True,False,...,12590229,屏東市,NaN,510.000,統棒門市,922980.000,SevenEleven,Finish,nan,NaN
5197282,20170303,4242056,b'JV\x97\x98\x9a\xf7\xe0\xee\xa4(\x00c\xc6\x17...,AndroidApp,Mobile,Family,30337499,2896293,True,False,...,10399110,彰化縣,員林市,510.000,全家員林車頭店,9336.000,Family,Cancel,nan,NaN
4841342,20161123,2892204,b'k\xc4\x0b\x90\xcc\xb95\x02\x14\xa3\xb7\xca\x...,Web,Mobile,Family,24355121,2699540,True,False,...,8232804,彰化縣,員林市,510.000,全家員林黎明店,13537.000,Family,Cancel,nan,NaN
4052948,20160904,2851444,b'$\x1eY\x19_97#i\r\x05\x0f\xb8\xf8\x04^WK\xb7...,Web,Mobile,SevenEleven,19365277,2193373,True,False,...,6507766,彰化縣,員林市,510.000,員慶門市,130877.000,SevenEleven,Finish,nan,NaN
